In [1]:
import string
import pandas as pd
import ast

import nltk
from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize
from nltk import pos_tag

from sklearn.base import BaseEstimator, TransformerMixin

### REQUIRED (RUN ONCE/DOWNLOAD): word tag file for lemmatization
# nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rwidjaya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Function to read data

In [4]:
def process_file(filepath, headers):
    
    '''
    Takes a .txt file from the Cornell Movie Diaglogs Corpus 
    and returns a pandas dataframe
    
    '''
    lines = []
    
    to_longform  = ['text', 'line_id', 'genre']
    idx_col, tgt_col = (headers[:-1], headers[-1])
    
    with open(filepath, 'r', encoding = 'iso-8859-1') as f:
        for line in f:
            cols = line.split(' +++$+++ ')
            lines.append(cols)
            
    df = pd.DataFrame(lines, columns = headers)
    
    if tgt_col in to_longform:
        if tgt_col != 'text':
            df[tgt_col] = df[tgt_col].apply(lambda x: ast.literal_eval(x))
        
        else:
            ## CODE FROM NLTK PROCESSING BELOW
            prepos = text_preprocess()
            df[tgt_col] = df[tgt_col].apply(lambda x: prepos.tokenize(x))
            
        
        # Convert wide format column to it's own rows/long form
        df = df.set_index(idx_col)[tgt_col].apply(pd.Series).stack()
        df = df.reset_index()
        
        # Rename columns 
        add_idx_col = [tgt_col+'_idx', tgt_col]
        df.columns = idx_col + add_idx_col
        
    return df

In [20]:
class text_preprocess(BaseEstimator, TransformerMixin):

    def __init__(self, stopwords=None, punct=None,
                 lower=True, strip=True):
        self.lower      = lower
        self.strip      = strip
        self.stopwords  = stopwords or set(sw.words('english'))
        self.punct      = punct or set(string.punctuation)
        self.lemmatizer = WordNetLemmatizer()

    def fit(self, sent, y=None):
        return self

    # def inverse_transform(self, sent):
    #    return [" ".join(w) for w in sent]

    def transform(self, sent):
        return list(self.tokenize(sent))
        
    
    def tokenize(self, sent):
        # Break the sentence into part of speech tagged tokens
        ## print(pos_tag(wordpunct_tokenize(sent)))
        for token, tag in pos_tag(wordpunct_tokenize(sent)):
            # Apply preprocessing to the token
            token = token.lower() if self.lower else token
            token = token.strip() if self.strip else token
            token = token.strip('_') if self.strip else token
            token = token.strip('*') if self.strip else token
            
            # If stopword, ignore token and continue
            if token in self.stopwords:
                continue

            # If punctuation, ignore token and continue
            if all(char in self.punct for char in token):
                continue

            # Lemmatize the token and yield
            lemma = self.lemmatize(token, tag)
            yield lemma
    
    # Takes the word tag to access the appropriate WordNet dict to lemmatize word token,
    # if lemmatized word cannot be found in the folders, funct will treat it as a noun
    def lemmatize(self, token, tag):
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(tag[0], wn.NOUN)

        return self.lemmatizer.lemmatize(token, tag)

In [18]:
# Testing the text_preprocess class
test_text = "Harry This ain't he endless ...blonde babble. I'm like, boring myself."
a = text_preprocess()
b = a.transform(test_text) # The second elements of the tuple are the tags

[('Harry', 'NNP'), ('This', 'DT'), ('ain', 'NN'), ("'", "''"), ('t', 'NN'), ('he', 'PRP'), ('endless', 'VBD'), ('...', ':'), ('blonde', 'NN'), ('babble', 'JJ'), ('.', '.'), ('I', 'PRP'), ("'", "''"), ('m', 'NN'), ('like', 'IN'), (',', ','), ('boring', 'VBG'), ('myself', 'PRP'), ('.', '.')]


In [19]:
b

['harry', 'endless', 'blonde', 'babble', 'like', 'bore']

## Read character metadata

In [15]:
headers = ['character_id', 'name', 'movie_id', 'movie_title', 'gender', 'position']

file = 'cornell_movie_dialogs_corpus/movie_characters_metadata.txt'

characters = process_file(file, headers)

In [16]:
characters.head()

,character_id,name,movie_id,movie_title,gender,position
0,u0,BIANCA,m0,10 things i hate about you,f,4\n
1,u1,BRUCE,m0,10 things i hate about you,?,?\n
2,u2,CAMERON,m0,10 things i hate about you,m,3\n
3,u3,CHASTITY,m0,10 things i hate about you,?,?\n
4,u4,JOEY,m0,10 things i hate about you,m,6\n


In [5]:
characters.groupby('gender').count()

,character_id,name,movie_id,movie_title,position
gender,,,,,
?,6020,6020,6020,6020,6020
F,45,45,45,45,45
M,150,150,150,150,150
f,921,921,921,921,921
m,1899,1899,1899,1899,1899


## Clean character metadata

In [6]:
characters.loc[characters.gender == 'F', 'gender'] = "f"
characters.loc[characters.gender == 'M', 'gender'] = "m"

In [7]:
characters.groupby('gender').count()

,character_id,name,movie_id,movie_title,position
gender,,,,,
?,6020,6020,6020,6020,6020
f,966,966,966,966,966
m,2049,2049,2049,2049,2049


## Read conversation metadata

In [6]:
# This takes pretty LONG (~5-10 mins), looking into multiprocessing 
file_lines = 'cornell_movie_dialogs_corpus/movie_lines.txt'

header_lines = ['line_id', 'char_id', 'movie_id', 'char_name','text']

lines = process_file(file_lines, header_lines)

In [14]:
len(lines)

1485913

In [13]:
lines.head()

,line_id,char_id,movie_id,char_name,text_idx,text
0,L985,u0,m0,BIANCA,0,hope
1,L984,u2,m0,CAMERON,0,okay
2,L925,u0,m0,BIANCA,0,let
3,L925,u0,m0,BIANCA,1,go
4,L924,u2,m0,CAMERON,0,wow


## Read lines metadata

In [38]:
## NOTE: Original text is curently not saved, can modify the process_file if it's needed

file_lines = 'cornell_movie_dialogs_corpus/movie_lines.txt'

header_lines = ['line_id', 'char_id', 'movie_id', 'char_name','text']

lines = process_file(file_lines, header_lines)

In [39]:
lines.head()

,line_id,char_id,movie_id,char_name,text_idx,text
0,L1045,u0,m0,BIANCA,0,They
1,L1045,u0,m0,BIANCA,1,do
2,L1045,u0,m0,BIANCA,2,not
3,L1045,u0,m0,BIANCA,3,!
4,L1044,u2,m0,CAMERON,0,They


## Read title metadata

In [41]:
file_titles = 'cornell_movie_dialogs_corpus/movie_titles_metadata.txt'

header_titles = ['movie_id', 'movie_title', 'movie_year', 'imdb_rating', 'imdb_vote', 'genre']

titles = process_file(file_titles, header_titles)

In [42]:
titles.head()

,movie_id,movie_title,movie_year,imdb_rating,imdb_vote,genre_idx,genre
0,m0,10 things i hate about you,1999,6.90,62847,0,comedy
1,m0,10 things i hate about you,1999,6.90,62847,1,romance
2,m1,1492: conquest of paradise,1992,6.20,10421,0,adventure
3,m1,1492: conquest of paradise,1992,6.20,10421,1,biography
4,m1,1492: conquest of paradise,1992,6.20,10421,2,drama
